## Apress - Industrialized Machine Learning Examples

Andreas Francois Vermeulen
2019

### This is an example add-on to a book and needs to be accepted as part of that copyright.

## Chapter-010-03-Distributed-Evolutionary-Algorithms-Loop

## Using Distributed Evolutionary Algorithms

Install the library:

In [1]:
#!conda install -c conda-forge deap

## Part A - Import the libraries:

In [2]:
import operator
import random
import numpy
from deap import base
from deap import benchmarks
from deap import creator
from deap import tools

## Part B - Set Creator's Fitness Maximum

In [3]:
creator.create("FitnessMax", base.Fitness, weights=(1.0,))

## Part C - Activate Creator's Particle List

In [4]:
creator.create("Particle", list, fitness=creator.FitnessMax, speed=list, 
    smin=None, smax=None, best=None)

## Part D - Create function to create a particle

In [5]:
def generate(size, pmin, pmax, smin, smax):
    part = creator.Particle(random.uniform(pmin, pmax) for _ in range(size)) 
    part.speed = [random.uniform(smin, smax) for _ in range(size)]
    part.smin = smin
    part.smax = smax
    return part

## Part E - Create function to update particle

In [6]:
def updateParticle(part, best, phi1, phi2):
    u1 = (random.uniform(0, phi1) for _ in range(len(part)))
    u2 = (random.uniform(0, phi2) for _ in range(len(part)))
    v_u1 = map(operator.mul, u1, map(operator.sub, part.best, part))
    v_u2 = map(operator.mul, u2, map(operator.sub, best, part))
    part.speed = list(map(operator.add, part.speed, map(operator.add, v_u1, v_u2)))
    for i, speed in enumerate(part.speed):
        if speed < part.smin:
            part.speed[i] = part.smin
        elif speed > part.smax:
            part.speed[i] = part.smax
    part[:] = list(map(operator.add, part, part.speed))


## Part F - Setup the Evolution Toolbox

In [7]:
toolbox = base.Toolbox()
toolbox.register("particle", generate, size=2, pmin=-6, pmax=6, smin=-3, smax=3)
toolbox.register("population", tools.initRepeat, list, toolbox.particle)
toolbox.register("update", updateParticle, phi1=2.0, phi2=2.0)
toolbox.register("evaluate", benchmarks.h1)

## Part G - Setup Main Function

In [8]:
def main_proc():
    pop = toolbox.population(n=5)
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", numpy.mean)
    stats.register("std", numpy.std)
    stats.register("min", numpy.min)
    stats.register("max", numpy.max)

    logbook = tools.Logbook()
    logbook.header = ["gen", "evals"] + stats.fields

    GEN = 1000
    best = None

    for g in range(GEN):
        for part in pop:
            part.fitness.values = toolbox.evaluate(part)
            if not part.best or part.best.fitness < part.fitness:
                part.best = creator.Particle(part)
                part.best.fitness.values = part.fitness.values
            if not best or best.fitness < part.fitness:
                best = creator.Particle(part)
                best.fitness.values = part.fitness.values
        for part in pop:
            toolbox.update(part, best)

        # Gather all the fitnesses in one list and print the stats
        logbook.record(gen=g, evals=len(pop), **stats.compile(pop))
        #print(logbook.stream)
    
    return pop, logbook, best

## Part H - Run the Solution

In [9]:
%%javascript
IPython.OutputArea.auto_scroll_threshold = 9999;

<IPython.core.display.Javascript object>

In [10]:
for i in range(20):
    pop, logbook, best = main_proc()
    print('### Population - %3d ###################################'% (i+1))
    for j in range(len(pop)):
        print(pop[j])
    print('### Best ###############################################')
    print(best)
    print('########################################################\n')

### Population -   1 ###################################
[8.005238570437282, 6.387535742634365]
[10.892026420766413, 10.34560744090591]
[7.558018911651953, 7.2673396442064]
[7.361325536253393, 4.3150773644303255]
[8.918372770729226, 8.80177753379216]
### Best ###############################################
[8.65924711050977, 6.7686715147140815]
########################################################

### Population -   2 ###################################
[6.2191249396380925, 8.415043098699506]
[10.745792969963578, 4.6439668220389585]
[8.225084670580717, 6.1059894003114366]
[8.139017049779218, 6.832242949081131]
[7.0509921634702035, 5.520388151566137]
### Best ###############################################
[8.700443707448319, 6.746432529674803]
########################################################

### Population -   3 ###################################
[10.39764565434325, 4.06732519586465]
[9.196086055989403, 7.858910229589152]
[8.745900276024702, 5.292139933313779]
[9.99860180

## Done

In [11]:
import datetime
now = datetime.datetime.now()
print('Done!',str(now))

Done! 2019-04-27 12:29:51.604508
